In [76]:
import numpy as np
import pandas as pd
import os

In [77]:
cwd = str(os.getcwd()) #Get the current working directory
path_of_recordings = cwd + '/recordings/' #Path where all the recordings are stored
raw_labels = np.genfromtxt(os.path.join(cwd, 'filename_and_label.txt'), dtype=np.str)#Load the file containing label corresponding to file name

In [78]:
X = [] #X is a list with each element a recording
Y = [] #Y is a list with each element being label of that recording
for r, d, f in os.walk(path_of_recordings):
    for file in f:
        if file.endswith(".txt"): #To avoid any other file than '.txt'
            path_of_file = os.path.join(r, file)
            print(path_of_file)#Print the global path of each loaded file
            raw_data = np.loadtxt(path_of_file)#Load the raw data
            X.append(raw_data)#Append the loaded array to the list
            index = np.where(raw_labels[0,:]==file)#Find the label corresponding to loaded file
            Y.append(int(raw_labels[index][:,1]))#Append the label

/Users/Anonymous/Desktop/OCET/recordings/recording_file_a.txt


In [79]:
sample_data = X[0]#Load the first element of list
time_stamp = sample_data[0,:]#Slice off the time stamps
data = sample_data[1:,:]#Slice the data
print(time_stamp.shape)
print(data.shape)
print(Y[0])

(27000,)
(30, 27000)
0
